# Brain Spark : Octomap

## 1. Space of map saved on disk comparison

In [1]:
import numpy as np
import open3d as o3d
import os
from scripts.regular_voxel_grid import RegularVoxelGrid, visualize_voxel_grid

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
voxel_grid = RegularVoxelGrid(resolution=0.25, max_coord=10, min_coord=-10)

In [4]:
voxel_grid.create_regular_voxel_map()
visualize_voxel_grid(voxel_grid.get_voxel_grid_points())

In [23]:
voxel_grid.save_voxel_grid_ply()
print(f"Saved voxel map representation space : {voxel_grid.get_saved_voxel_map_disk_size()} MB")

Saved voxel map representation space : 1.0029869079589844 MB


In [7]:
voxel_grid.write_voxel_grid_points_to_txt()


### Creating the octomap in CPP and saving it to disk

In [19]:
# Create an Open3D Octree for visualization first
octree = o3d.geometry.Octree(max_depth=8)  # The tree is stopped when the depth level reaches 8
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(voxel_grid.get_voxel_grid_points())
octree.convert_from_point_cloud(pcd, size_expand=0.02)

# Visualize the Octree
o3d.visualization.draw_geometries([octree])

Since open3d does not allow us to save the octree, I am going to use the official Octomap repository to create the same octomap using the above points, and then save it to a usable binary file. And then check the saved file size. 

In [20]:
!cp simple_example.cpp OCTOMAP/octomap/src
if not os.path.exists('OCTOMAP/build'):
    os.makedirs('OCTOMAP/build', exist_ok=True)
!cd OCTOMAP/build && cmake .. && make
!./OCTOMAP/bin/simple_example  # will create the octomap corresponding to above voxel grid points and save it to a binary file

CMake Deprecation Warning at CMakeLists.txt:1 (CMAKE_MINIMUM_REQUIRED):
  Compatibility with CMake < 3.5 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.


-- 

-- octomap building as Release
-- Could NOT find Doxygen (missing: DOXYGEN_EXECUTABLE) 
-- 

-- Compile octomap using: make
-- Install octomap using: make install
--     (be sure to set the correct CMAKE_INSTALL_PREFIX before)
-- Compile API-documentation using: make docs

-- 

-- octovis building as Release
-- Found octomap version: 1.10.0
-- Could NOT find OpenGL (missing: OPENGL_opengl_LIBRARY OPENGL_glx_LIBRARY OPENGL_INCLUDE_DIR) 
-- QGLViewer includes found in /home/sambaran.ghosal/BrainCorp/Sambaran/development/octomap/OCTOMAP/octovis/src/extern/QGLViewer
-- QGLViewer library not found.
-- Trying to build libQGLViewer from source in /home/sambaran.ghosal/BrainCorp/Sambaran

In [24]:
print(f"Saved octomap space: {os.path.getsize('OCTOMAP/bin/octree_from_point_cloud.bt') / (1024 * 1024)} MB")

Saved octomap space: 0.0643758773803711 MB


In [22]:
# Visualize the octomap
!octovis OCTOMAP/bin/octree_from_point_cloud.bt

Reading binary octree type OcTree


## 2. Dynamically adding points comparison

In [25]:
### Another advantage of using octomap is that we can dynamically add points to the octomap and update it, no matter if the current point being added is very far from the existing points. In a regular voxel grid, if we want to add a point that is out of bounds from the initialized voxel grid space, we need to reinitialize the voxel grid with a larger space. This would require us to reinitialize the voxel grid and save it to disk again.

In [27]:
# Add a point to the voxel grid
voxel_grid.add_point_to_voxel_grid(np.array([20, -6, 0]))  # This point is out of bounds of the current voxel grid space. Hence trying to add will result in an error

IndexError: index 120 is out of bounds for axis 0 with size 80

### Adding a point to the octomap 

#### Modifying simple_example.cpp to add a point to the octomap that is far away from existing points 

In [17]:
!cp simple_example.cpp OCTOMAP/octomap/src
!cd OCTOMAP/build && cmake .. && make
!./OCTOMAP/bin/simple_example
!octovis octree_from_point_cloud.bt

CMake Deprecation Warning at CMakeLists.txt:1 (CMAKE_MINIMUM_REQUIRED):
  Compatibility with CMake < 3.5 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.


-- 

-- octomap building as Release
-- 

-- Compile octomap using: make
-- Install octomap using: make install
--     (be sure to set the correct CMAKE_INSTALL_PREFIX before)
-- Compile API-documentation using: make docs

-- 

-- octovis building as Release
-- Found octomap version: 1.10.0
-- Could NOT find OpenGL (missing: OPENGL_opengl_LIBRARY OPENGL_glx_LIBRARY OPENGL_INCLUDE_DIR) 
-- QGLViewer includes found in /home/sambaran.ghosal/BrainCorp/Sambaran/development/octomap/OCTOMAP/octovis/src/extern/QGLViewer
-- QGLViewer library not found.
-- Trying to build libQGLViewer from source in /home/sambaran.ghosal/BrainCorp/Sambaran/development/octomap/OCTOMAP/octovis/src/extern/QGLViewer

### As we can see, the octomap is updated with the new point that is far away from the existing points. This is not possible with a regular voxel grid.

## 3. Adaptive resolution comparison

### Octomap has the ability to adaptively change the resolution of the octree based on the points being added. This is not possible with a regular voxel grid.

In [5]:
voxel_grid = RegularVoxelGrid(resolution=0.25, max_coord=5, min_coord=-5)
# Generate dense points in a small area (high resolution needed)
dense_points = np.random.rand(1000, 3)

# Generate sparse points in a larger area (low resolution needed)
sparse_points = (np.random.rand(10, 3) * 2) + 2.5

# Combine dense and sparse points
points = np.vstack((dense_points, sparse_points))
voxel_grid.add_point_to_voxel_grid(points)

In [6]:
visualize_voxel_grid(voxel_grid.get_voxel_grid_points())

In [8]:
voxel_grid.write_voxel_grid_points_to_txt(filename="adaptive_voxel_grid_points.txt")

In [9]:
voxel_grid.save_voxel_grid_ply()
print(f"Saved voxel map representation space : {voxel_grid.get_saved_voxel_map_disk_size()} MB")

Saved voxel map representation space : 0.4395942687988281 MB


In [12]:
!cd OCTOMAP/build && cmake .. && make
!./OCTOMAP/bin/simple_example
!octovis octree_from_point_cloud.bt

CMake Deprecation Warning at CMakeLists.txt:1 (CMAKE_MINIMUM_REQUIRED):
  Compatibility with CMake < 3.5 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.


-- 

-- octomap building as Release
-- 

-- Compile octomap using: make
-- Install octomap using: make install
--     (be sure to set the correct CMAKE_INSTALL_PREFIX before)
-- Compile API-documentation using: make docs

-- 

-- octovis building as Release
-- Found octomap version: 1.10.0
-- Could NOT find OpenGL (missing: OPENGL_opengl_LIBRARY OPENGL_glx_LIBRARY OPENGL_INCLUDE_DIR) 
-- QGLViewer includes found in /home/sambaran.ghosal/BrainCorp/Sambaran/development/octomap/OCTOMAP/octovis/src/extern/QGLViewer
-- QGLViewer library not found.
-- Trying to build libQGLViewer from source in /home/sambaran.ghosal/BrainCorp/Sambaran/development/octomap/OCTOMAP/octovis/src/extern/QGLViewer

In [17]:
print(f"Saved octomap space: {os.path.getsize('octree_from_point_cloud.bt') / (1024 * 1024)} MB")

Saved octomap space: 2.7448387145996094 MB


## Ray Casting

### Ray casting is a method to mark free spaces along a ray from the sensor origin to the end of the ray. This is useful for collision detection and path planning. Octomap provides a ray casting method to check if a ray is free from obstacles. This is not possible with a regular voxel grid. During octomapping, when we insert ray casting between sensor origin and the detected endpoint from the lidar, we can mark free spaces in the octomap. 

### Ray casting can also be used to see if a ray starting from a point, in a direction intersects with some obstacle or not within a max range. This is useful for path planning and collision detection.

### Modify simple_example.cpp to do ray casting during octomap to insert free cells during adding points from pointcloud. Then in octovis press F to view the free space pop up. 

In [23]:
!cp simple_example.cpp OCTOMAP/octomap/src
!cd OCTOMAP/build && cmake .. && make
!./OCTOMAP/bin/simple_example
!octovis octree_from_point_cloud.bt

CMake Deprecation Warning at CMakeLists.txt:1 (CMAKE_MINIMUM_REQUIRED):
  Compatibility with CMake < 3.5 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.


-- 

-- octomap building as Release
-- 

-- Compile octomap using: make
-- Install octomap using: make install
--     (be sure to set the correct CMAKE_INSTALL_PREFIX before)
-- Compile API-documentation using: make docs

-- 

-- octovis building as Release
-- Found octomap version: 1.10.0
-- Could NOT find OpenGL (missing: OPENGL_opengl_LIBRARY OPENGL_glx_LIBRARY OPENGL_INCLUDE_DIR) 
-- QGLViewer includes found in /home/sambaran.ghosal/BrainCorp/Sambaran/development/octomap/OCTOMAP/octovis/src/extern/QGLViewer
-- QGLViewer library not found.
-- Trying to build libQGLViewer from source in /home/sambaran.ghosal/BrainCorp/Sambaran/development/octomap/OCTOMAP/octovis/src/extern/QGLViewer

### Add code to do ray casting for collision detection along a specific direction